In [6]:
import pandas as pd
import numpy as np
import smtplib
import bs4
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# # Setting browser
# Setting browser's options
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--start-maximized")
options.add_argument("--disable-popup-blocking")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(0)
print("-- Driver setup and browser opened")

# Access link
sleep(1)
driver.get('https://glints.com/vn/opportunities/jobs/explore')
print('-- Glints accessed')

# # Surpass website's noti
# Scroll page to pop up the message
sleep(7)
html = driver.find_element(By.TAG_NAME,'html')
html.send_keys(Keys.DOWN)

# Escape
sleep(5)
html.send_keys(Keys.ESCAPE)

# Extract data function
# FUNCTION
def extract(link):
    element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="GlintsContainer-sc-ap1z3q-0 iUnyrV"]')))
    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
    soup = BeautifulSoup(html_of_interest, 'lxml')

    item_dict = {}

    # Company Name
    try:
        item_dict['Name'] = soup.select('div[class="TopFoldsc__JobOverviewHeader-sc-kklg8i-22 ihxBLZ"]')[0].text
    except:
        item_dict['Name'] = None 

    # Company Name
    try:
        item_dict['Company'] = soup.select('div[class="TopFoldsc__JobOverViewCompanyName-sc-kklg8i-5 eLQvRY"]>a')[0].text
    except:
        item_dict['Company'] = None 

    # Location
    try:
        item_dict['Location'] = soup.select('div[class="TopFoldsc__JobOverViewCompanyLocation-sc-kklg8i-6 gLATOW"]>span>a')[0].text
    except:
        item_dict['Location'] = None 

    info = soup.select('div[class="TopFoldsc__JobOverViewInfo-sc-kklg8i-9 EWOdY"]')
    info_list = []
    for ite in info:
        info_list.append(ite.text)
    # Salary range 
    try:
        salary_ele = soup.select('div[class="TopFoldsc__JobOverViewInfoContainer-sc-kklg8i-8 fgSCsF"]>div>span')[0]
        salary_txt = salary_ele.text 
        salary = salary_txt.split('D')[1].split('/')[0]
        info_list.remove(salary_txt)
        item_dict['Salary'] = salary
    except:
        item_dict['Salary']= None 
        
    # Experienc, Type, Field`
    item_dict['Experience'] = None
    item_dict['Type'] = None 

    for ele in info_list:
        if 'kinh nghiệm' in ele:
            item_dict['Experience'] = ele 
            info_list.remove(ele)
    for ele in info_list:  
        if ('Việc' in ele) or ('Thực Tập' in ele):
            item_dict['Type'] = ele
            info_list.remove(ele)
    try:
        item_dict['Field'] = info_list[0]
    except:
        item_dict['Field'] = None

    # Posted and Updated time
    try:
        posted = soup.select('span[class="TopFoldsc__PostedAt-sc-kklg8i-11 eRKLIR"]')
        posted = posted[0].text.split(' ',1)[1]
        item_dict['Posted'] = posted
    except:
        item_dict['Posted'] = None

    try:
        updated = soup.select('span[class="TopFoldsc__UpdatedAt-sc-kklg8i-12 bYndtI"]')
        updated = updated[0].text.split(' ',1)[1]
        item_dict['Updated'] = updated
    except:
        item_dict['Updated'] = None

    #  Skill required
    try:
        skills = soup.select('div[class="TagStyle__TagContainer-sc-66xi2f-1 gtZZMG aries-tag Skillssc__TagOverride-sc-11imayw-3 fyJqX"]')
        skills_str = ''
        for i in range(len(skills)):
            if i == (len(skills) -1 ):
                sub = skills[i].text
            else:
                sub = skills[i].text + ', '
            skills_str = skills_str + sub
        item_dict['Skill Required'] = skills_str
    except:
        item_dict['Skill Required'] = None 

    # Link
    try:
        item_dict['Link'] = link
    except:
        item_dict['Link'] = None 
    df_list.append(item_dict)


# Import url from detail_urls
#  import urls from the file
urls = pd.read_csv('detail_urls.csv')
urls = urls['Detail Urls'][709:]

# Begin to scrap data
print('-- Process:',end= ' ')
sleep_time = 5
try:
    cnt = 0
    df_list = []
    total_urls = len(urls)

    # Loop through detail_urls to get link
    for link in urls.iloc:
        driver.get(link)
        sleep(sleep_time)
        try: 
            extract(link)
        except Exception as e:
            pass 
            
        cnt += 1
        # PROGRESS ANNOUNCEMENT
        if cnt%10 == 0:
            print(f'{round((cnt/total_urls)*100,3)}%', end=' --> ')

    # Notification
    print(f'-- Successfully scraped all data!!')

except Exception as err:
    #  Notification
    print(f'-- Error raised at url number {cnt}, program stopped')
    print(e.__class__, "occurred.")
    break


# Save df to data file named ScrapedData
df = pd.DataFrame(df_list)
df.to_csv('ScrapedData2.csv')



-- Driver setup and browser opened
-- Glints accessed
-- Process: 